![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/MENTAL_HEALTH.ipynb)

## **Mental Health Models**

## **Colab Setup**

In [27]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [28]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [29]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.8
Spark NLP_JSL Version : 4.2.8


# **General Function for MedicalBertForSequenceClassification Pipeline**





In [30]:
def run_pipeline(model, text, lang = 'en'):  
  document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

  tokenizer = Tokenizer() \
    .setInputCols("document") \
    .setOutputCol("token")

  sequenceClassifier = MedicalBertForSequenceClassification.pretrained(model, lang, "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class_")

  pipeline = Pipeline(
      stages=[
          document_assembler, 
          tokenizer,
          sequenceClassifier
      ])

  df = spark.createDataFrame(text, StringType()).toDF("text")
  result = pipeline.fit(df).transform(df)
   
  print("\n")
  print("<----------------- MODEL NAME:","\033[1m" + model + "\033[0m"," ----------------- >")
  
  res = result.select(F.explode(F.arrays_zip(result.document.result, 
                                             result.class_.result,
                                             result.class_.metadata)).alias("col"))\
               .select(F.expr("col['1']").alias("prediction"),
                       F.expr("col['2']").alias("confidence"),
                       F.expr("col['0']").alias("sentence"))
                  
  if res.count()>1:
    udf_func = F.udf(lambda x,y:  x["Some("+str(y)+")"])
    print("\n",model,"\n") 
    res.withColumn('confidence', udf_func(res.confidence, res.prediction)).show(truncate=False)

# **MODELS**

## **bert_sequence_classifier_depression**

In [31]:
model = "bert_sequence_classifier_depression"

In [32]:
sample_texts = [
"""I definitely know my karaoke song now so hmu and it's by Britney shocking""", 
"""Johnny Cash going on day three sleeping in my bed. I must be severely depressed cus he can sense it.""", 
"""My mom said she was going to take me to the hospital so I started freaking out. Threw up on the living room floor.  Not much the hospital can do for me so what’s the point of going?""",
"""wow i’m  but i have so many good things happening in the next few months i just gotta hang in there.......""",
"""Feeling okish one minute then ?? for no reason what's so ever. Psychiatrist was right chronic depression gives no warnings, mood just dips when it feels like it.  Top it off anxiety is at a high too.  """]

In [33]:
run_pipeline(model, sample_texts)

bert_sequence_classifier_depression download started this may take some time.
[OK!]


<----------------- MODEL NAME: bert_sequence_classifier_depression  ----------------- >

 bert_sequence_classifier_depression 

+---------------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction     |confidence|sentence                                                                                                                                                                                                 |
+---------------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|no-depression  |0.9999039 |I definitely know my karaoke song now so hmu and it's by Britney 

## **bert_sequence_classifier_depression_binary**

In [34]:
model = "bert_sequence_classifier_depression_binary"

In [35]:
sample_texts = [
"""I definitely know my karaoke song now so hmu and it's by Britney shocking""", 
"""Johnny Cash going on day three sleeping in my bed. I must be severely depressed cus he can sense it.""", 
"""My mom said she was going to take me to the hospital so I started freaking out. Threw up on the living room floor.  Not much the hospital can do for me so what’s the point of going?""",
"""wow i’m  but i have so many good things happening in the next few months i just gotta hang in there.......""",
"""Feeling okish one minute then ?? for no reason what's so ever. Psychiatrist was right chronic depression gives no warnings, mood just dips when it feels like it.  Top it off anxiety is at a high too.  """]


In [36]:
run_pipeline(model, sample_texts)

bert_sequence_classifier_depression_binary download started this may take some time.
[OK!]


<----------------- MODEL NAME: bert_sequence_classifier_depression_binary  ----------------- >

 bert_sequence_classifier_depression_binary 

+-------------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction   |confidence|sentence                                                                                                                                                                                                 |
+-------------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|no-depression|0.99752676|I definitely know my karaoke song now so hmu and it'

## **bert_sequence_classifier_depression_twitter**

In [37]:
model = "bert_sequence_classifier_depression_twitter"

In [38]:
sample_texts = [
"""I definitely know my karaoke song now so hmu and it's by Britney shocking""", 
"""With ageing populations at risk of  and  (and depression's myriad health effects so higher healthcare costs), as well as rising childcare costs, approaches like intergenerational care seem beautifully cost efficient and solve 1 problem with another problem. pic.twitter.com/QouBTSRhR6""", 
"""My mom said she was going to take me to the hospital so I started freaking out. Threw up on the living room floor.  Not much the hospital can do for me so what’s the point of going?""",
"""wow i’m  but i have so many good things happening in the next few months i just gotta hang in there.......""",
"""Feeling okish one minute then ?? for no reason what's so ever. Psychiatrist was right chronic depression gives no warnings, mood just dips when it feels like it.  Top it off anxiety is at a high too.  """]

In [39]:
run_pipeline(model, sample_texts)

bert_sequence_classifier_depression_twitter download started this may take some time.
[OK!]


<----------------- MODEL NAME: bert_sequence_classifier_depression_twitter  ----------------- >

 bert_sequence_classifier_depression_twitter 

+-------------+----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction   |confidence|sentence                                                                                                                                                                                                                                                                                    |
+-------------+----------+----------------------------------------------------------------------------------------------------------------